## Sorghum -100 Cultivar Identification with CNN

## Import Packages

In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import glob

## Configuration

In [ ]:
class Config:
    image_size = 128
    batch_size = 128
    dataset_name = "sorghum-id-fgvc-9"
    output_dataset_name = "sorghum-100-cultivar-identification-output"
    is_training = False
config = Config()

## Import dataset

In [ ]:
train = pd.read_csv(f"../input/{config.dataset_name}/train_cultivar_mapping.csv")
train.head()

## Distribution of Sample Count for different categories

In [ ]:
train.cultivar.value_counts().hist()

## Number of Classes

In [ ]:
unique_cultivars = list(train.cultivar.unique())
num_classes = len(unique_cultivars)
num_classes

In [ ]:
train["file_path"] = train["image"].apply(lambda image: f"../input/{config.dataset_name}/train_images/" + image)
train["cultivar_index"] = train["cultivar"].map(lambda item: unique_cultivars.index(item))
train["is_exist"] = train["file_path"].apply(lambda file_path: os.path.exists(file_path))
train = train[train.is_exist==True]
train.head()

## Modeling

In [ ]:
def preprocess(image_url, target):
    image_string = tf.io.read_file(image_url)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.central_crop(image, 1.0)
    image = tf.image.resize(image, (config.image_size, config.image_size))
    return image, target

In [ ]:
def block(x, filters, kernel_size, repetitions, pool_size=2, strides=2):
    for i in range(repetitions):
        x = tf.keras.layers.Conv2D(filters, kernel_size, activation='relu', padding='same')(x)
    x = tf.keras.layers.MaxPooling2D(pool_size, strides)(x)
    return x

def get_model():
    inputs = tf.keras.Input((config.image_size, config.image_size , 3))
    x = block(inputs, 8, 3, 2)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = block(x, 16, 3, 2)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = block(x, 32, 3, 2)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = block(x, 64, 3, 2)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = block(x, 128, 3, 2)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    output = tf.keras.layers.Dense(num_classes, activation="softmax")(x)
    model = tf.keras.Model(inputs=inputs, outputs=output)
    model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

Let's take a look at this Model:

In [ ]:
model = get_model()
model.summary()
tf.keras.utils.plot_model(model, show_shapes=True)

## Model Training

In [ ]:
tf.keras.backend.clear_session()
models = []
historys = []
is_k_fold = False
if is_k_fold:
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    best_fold = 0
    for fold, (train_indices, val_indices) in enumerate(kfold.split(train, train["cultivar_index"])):
        if best_fold != None and fold != best_fold:
            continue
        train_df = train.iloc[train_indices]
        val_df= train.iloc[val_indices]
        checkpoint_path = f"model_{fold}.h5"
        checkpoint = tf.keras.callbacks.ModelCheckpoint(
            checkpoint_path, 
            save_best_only=True
        )
        early_stop = tf.keras.callbacks.EarlyStopping(
            min_delta=1e-4, 
            patience=10
        )
        reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
            factor=0.3,
            patience=2, 
            min_lr=1e-7
        )
        callbacks = [early_stop, checkpoint, reduce_lr]
        train_ds = tf.data.Dataset.from_tensor_slices((train_df["file_path"], train_df["cultivar_index"])).map(preprocess).shuffle(512).batch(config.batch_size).cache().prefetch(tf.data.AUTOTUNE)
        val_ds = tf.data.Dataset.from_tensor_slices((val_df["file_path"], val_df["cultivar_index"])).map(preprocess).batch(config.batch_size).cache().prefetch(tf.data.AUTOTUNE)
        model = get_model()
        if config.is_training:
            history = model.fit(train_ds, epochs=300, validation_data=val_ds, callbacks=callbacks)
            for metrics in [("loss", "val_loss"), ("accuracy", "val_accuracy")]:
                pd.DataFrame(history.history, columns=metrics).plot()
                plt.show()
            model.load_weights(checkpoint_path)
            historys.append(history)
        else:
            model.load_weights(f"../input/{config.output_dataset_name}/{checkpoint_path}")
        models.append(model)
else:
    checkpoint_path = "model.h5"
    model = get_model()
    if config.is_training:
        train_df, val_df = train_test_split(train, test_size=0.2, random_state=42)
        train_ds = tf.data.Dataset.from_tensor_slices((train_df["file_path"], train_df["cultivar_index"])).map(preprocess).shuffle(512).batch(config.batch_size).cache().prefetch(tf.data.AUTOTUNE)
        val_ds = tf.data.Dataset.from_tensor_slices((val_df["file_path"], val_df["cultivar_index"])).map(preprocess).batch(config.batch_size).cache().prefetch(tf.data.AUTOTUNE)
        checkpoint = tf.keras.callbacks.ModelCheckpoint(
            checkpoint_path, 
            save_best_only=True
        )
        early_stop = tf.keras.callbacks.EarlyStopping(
            min_delta=1e-4, 
            patience=10
        )
        reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
            factor=0.3,
            patience=2, 
            min_lr=1e-7
        )
        callbacks = [checkpoint]
        history = model.fit(train_ds, epochs=100, validation_data=val_ds, callbacks=callbacks)
        for metrics in [("loss", "val_loss"), ("accuracy", "val_accuracy")]:
            pd.DataFrame(history.history, columns=metrics).plot()
            plt.show()
        model.load_weights(checkpoint_path) 
    else:
        model.load_weights(f"../input/{config.output_dataset_name}/{checkpoint_path}")
    models.append(model)

## Submission

In [ ]:
def preprocess_test(image_url):
    image_string = tf.io.read_file(image_url)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.central_crop(image, 1.0)
    image = tf.image.resize(image, (config.image_size, config.image_size))
    return image, 0

def inference(models, test_ds):
    total_results = []
    for model in models:
        total_results.append(model.predict(test_ds))
    results = np.mean(total_results, axis=0)
    return np.argmax(results, axis=-1)

In [ ]:
%%time
if not config.is_training:
    submission = pd.read_csv(f"../input/{config.dataset_name}/sample_submission.csv")
    submission["file_path"] = submission["filename"].apply(lambda filename: f"../input/{config.dataset_name}/test/{filename}")
    test_ds = tf.data.Dataset.from_tensor_slices((submission["file_path"])).map(preprocess_test).batch(config.batch_size).cache().prefetch(tf.data.AUTOTUNE)
    y_pred = inference(models, test_ds)
    np_unique_cultivars = np.array(unique_cultivars) 
    cultivars = list(np_unique_cultivars[y_pred.reshape(-1)])
    submission["cultivar"] = cultivars
    submission[["filename", "cultivar"]].to_csv("submission.csv", index=False)